# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [2]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio


In [3]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [4]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [5]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [6]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
# split_documents = [
#     "Bananas are yellow and tasty in smoothies.",
#     "Mars rovers are equipped with AI navigation tools.",
#     "Shakespeare wrote many famous plays including Hamlet."
# ]
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [7]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [8]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?

##### ✅ Answer:

Yes, OpenAI provides a way to modify the embedding dimension when using the text-embedding-3-small model. By default, this model outputs a 1536-dimensional vector, but we can reduce this size by using the optional dimensions parameter in the API request.
For example, to get a smaller embedding (e.g., 256 dimensions), your API request would look like this:
{
      "model": "text-embedding-3-small",
      "input": "Input text here",
      "dimensions": 256
}
We can set the dimensions parameter to any integer between 1 and 1536, depending on your needs. Reducing the dimension can significantly improve performance in terms of speed, memory usage, and storage—especially for large-scale or latency-sensitive applications—while still retaining strong semantic performance, thanks to how the model is trained.

2. What technique does OpenAI use to achieve this?

##### ✅ Answer:

OpenAI uses a technique called Matryoshka Representation Learning (MRL) to enable flexible embedding sizes in the text-embedding-3-small and text-embedding-3-large models. This allows users to generate high-quality embeddings at reduced dimensions — such as 256, 512, or any value up to the default 1536 — without retraining the model or sacrificing much accuracy.

What is Matryoshka Representation Learning?

Matryoshka Representation Learning (MRL) is inspired by Russian nesting dolls, where smaller dolls fit perfectly inside larger ones. Similarly, in MRL:

The earliest dimensions of the embedding vector (e.g., the first 256 values) are trained to hold the most important semantic information.

As you add more dimensions (e.g., going from 256 → 512 → 1536), you add progressively more detail, but the core meaning is already captured early on.
This makes it possible to truncate embeddings to a smaller size while still retaining meaningful and useful representations.

How does it work during training?
OpenAI explicitly trains the embedding model to optimize partial vectors — that is, the prefixes of the full-length embedding are treated as standalone representations.

So instead of training just for one fixed-length vector:

The model learns to make smaller slices of the vector (e.g., first 128, 256, etc.) semantically strong.
This is done by applying special training objectives that encourage each subset of the vector to perform well on downstream tasks.

This is not simple truncation — it's a deliberate and efficient design choice baked into the training pipeline.

Why is this useful?
MRL enables powerful flexibility and real-world efficiency:

✅ Dynamic trade-offs:
Use 256 dimensions for fast, low-memory inference.

Use the full 1536 dimensions when you need top-tier semantic accuracy.

✅ No retraining needed:
You can switch between embedding sizes just by changing a parameter in the API.

No need to fine-tune or modify the model.

✅ Efficient storage and search:
Smaller embeddings reduce storage footprint, improve vector database speed, and optimize similarity search performance.

✅ Seamless integration:
Perfect for resource-constrained environments like mobile devices.

Compatible with scalable backends for large-scale search or RAG systems.

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [ ]:
# vector_db = VectorDatabase(distance_metric="euclidean")
# vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

##### Code Changes to test the difference between different distance metrics #####

query = "What is the Michael Eisner Memorial Weak Executive Problem?"
# query = "What does the text say about management styles?"
k = 3

for metric in ["cosine", "euclidean", "dot"]:
    print(f"\n Metric: {metric.upper()}")
    print("-" * 40)

    vector_db = VectorDatabase(distance_metric=metric)
    vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

    results = vector_db.search_by_text(query, k=k)
    for i, (text, score) in enumerate(results, 1):
        print(f"{i}. Score: {score:.4f}")
        print(f"   Text: {text[:150].strip()}...\n")



 Metric: COSINE
----------------------------------------
1. Score: 0.6539
   Text: ordingly.
Seventh, when hiring the executive to run your former specialty, be
careful you don’t hire someone weak on purpose.
This sounds silly, but y...

2. Score: 0.5036
   Text: m. They have areas where they are truly deXcient in judgment or skill set. That’s just life. Almost nobody is brilliant
at everything. When hiring and...

3. Score: 0.4814
   Text: ed?
In reality — as opposed to Marc’s warped view of reality — it will
be extremely helpful for Marc [if he were actually the CEO,
which he is not] to...


 Metric: EUCLIDEAN
----------------------------------------
1. Score: -0.8320
   Text: ordingly.
Seventh, when hiring the executive to run your former specialty, be
careful you don’t hire someone weak on purpose.
This sounds silly, but y...

2. Score: -0.9964
   Text: m. They have areas where they are truly deXcient in judgment or skill set. That’s just life. Almost nobody is brilliant
at everyt

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

##### ✅ Answer:

Using an async (asynchronous) approach to collect embeddings offers major advantages in terms of speed, efficiency, and scalability — especially when working with multiple texts or building production-grade systems. In a synchronous (sync) approach, each embedding request is sent one at a time. The program waits for the first request to finish before moving on to the next, which can be painfully slow when processing large batches.

In contrast, async allows you to send multiple requests at the same time without blocking the execution of your code. This means you can launch 10, 50, or even 100 embedding requests concurrently, and your program will continue running while the responses are being processed in the background. As a result, you save time and make better use of available resources — especially important when working with APIs that involve network latency.

For example, imagine you have 1,000 product descriptions and you want to embed them using OpenAI’s text-embedding-3-small model. With a synchronous approach, it might take several minutes to send and receive all 1,000 requests, one after another. But with an asynchronous approach, you can batch and fire off many of those requests in parallel — cutting the total time down to a fraction, depending on how many concurrent requests your environment and rate limits support.

Async is particularly useful in modern Python ecosystems where tools like asyncio, aiohttp, or frameworks like FastAPI are commonly used. These tools allow developers to build highly responsive systems — like search engines, chatbots, or document-processing pipelines — where latency and throughput actually matter. With async, your app won’t freeze or bottleneck while waiting for each embedding to complete.

In summary, using an async approach to collect embeddings improves speed, reduces latency, scales better with workload size, and allows for more responsive and efficient systems. It’s the preferred method when dealing with multiple embedding calls or real-time user interactions.


Ask ChatGPT


> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [10]:
# vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

##### ✅ Answer:

Yes, there are several ways to achieve more reproducible and consistent outputs when using the OpenAI API. While language models are inherently probabilistic, OpenAI provides parameters that allow developers to reduce randomness and make the output more deterministic, especially for testing, debugging, or production use cases.

1. seed – Sets the random seed for generation (available in newer models like gpt-4-turbo), making outputs deterministic for the same input and settings.

2. temperature – Lower values (especially 0) reduce randomness, making the model more predictable.

3. top_p – Use a fixed value like 1.0 to control sampling range and reduce output variability.

4. Consistent prompt – Even small differences in wording or formatting can lead to different outputs, so keep the prompt exactly the same.

5. Fixed model version – Always specify a model snapshot (e.g., gpt-4-0613) to avoid changes from future model updates.

6. max_tokens and stop – Limit output length and define clear stopping points to prevent unnecessary variation.
> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [11]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [12]:
print(response)

The best way to write a loop in Python often depends on what you want to achieve, but here are a few guidelines to ensure your loop is efficient, readable, and Pythonic:

### 1. **Use `for` Loops for Iterables**
When you need to iterate over a collection (like a list, tuple, or string), a `for` loop is usually the best choice. It’s clean and easy to understand. Here’s an example:

```python
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(fruit)
```

### 2. **Use `while` Loops for Conditional Iteration**
If you need to loop until a specific condition is met, a `while` loop is suitable. Make sure to update the condition within the loop to avoid infinite loops:

```python
count = 0
while count < 5:
    print(count)
    count += 1
```

### 3. **Use List Comprehensions**
For simple loops that create lists, consider using list comprehensions. They are concise and often quicker:

```python
squares = [x**2 for x in range(10)]
print(squares)
```

### 4. **Avoid Bare `break

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [13]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [14]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [15]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")


Response: The 'Michael Eisner Memorial Weak Executive Problem' refers to a situation where a CEO, who has a strong background in a specific function (such as product management, sales, or marketing), ends up hiring a weak executive in that same function. This often occurs as a result of the CEO's difficulty in letting go of the function that initially contributed to their success. The example given in the context is of Michael Eisner, the former CEO of Disney, who had been a successful TV network executive but saw the ABC network fall to fourth place after he acquired it. Eisner expressed a belief that he could personally turn around ABC if he had more time, which emphasizes the issue of executive overconfidence and the risks of surrounding oneself with weaker talent to maintain a sense of control or superiority in a formerly held role.

Context Count: 3
Similarity Scores: ['Source 1: 0.658', 'Source 2: 0.509', 'Source 3: 0.479']


#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

##### ✅ Answer:

To generate more thoughtful, detailed, and structured responses from a large language model (LLM), a highly effective prompting strategy is to encourage the model to reason step by step. This involves crafting the prompts in a way that guides the model to explain its logic or thought process before arriving at a final answer. For example, we can include phrases like:

“Let’s think step by step.”
“Explain your reasoning.”
“Break it down.”
“Walk through this logically.”

These types of cues prompt the model to simulate a human-like chain of reasoning, which typically leads to more accurate and well-explained answers — especially when dealing with complex, multi-step problems or tasks that involve logic, math, planning, or causal thinking.

This strategy is formally known as Chain-of-Thought (CoT) prompting.

Chain-of-Thought prompting works by unlocking the model’s ability to reason in natural language. Rather than providing a quick, surface-level answer, the model is encouraged to show its work — much like a student explaining how they solved a math problem. This makes the output not only more reliable but also easier to audit and understand.

Example:
Without Chain-of-Thought Prompting:

Q: If a pen costs $2 and a notebook costs $5, how much do three pens and two notebooks cost?
A: $16

With Chain-of-Thought Prompting:

Q: If a pen costs $2 and a notebook costs $5, how much do three pens and two notebooks cost? Let’s think step by step.
A: One pen costs $2, so three pens cost 3 × $2 = $6. One notebook costs $5, so two notebooks cost 2 × $5 = $10. Total cost is $6 + $10 = $16.

As shown, the second response is not only accurate but also transparent — it shows how the model arrived at the answer.

> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [ ]:
### YOUR CODE HERE

# Code Changes made in vectordatabase.py file

#vectordatabase.py

# class VectorDatabase:
#     def __init__(self, embedding_model: EmbeddingModel = None, distance_metric: str = "cosine"):
#         self.vectors = defaultdict(np.array)
#         self.embedding_model = embedding_model or EmbeddingModel()
#         self.distance_metric = distance_metric

#     def _cosine_similarity(self, vec1, vec2):
#         return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

#     def _euclidean_distance(self, vec1, vec2):
#         return -euclidean(vec1, vec2)  # negate so higher = more similar

#     def _dot_product(self, vec1, vec2):
#         return np.dot(vec1, vec2)

#     def _compute_similarity(self, vec1, vec2):
#         if self.distance_metric == "cosine":
#             return self._cosine_similarity(vec1, vec2)
#         elif self.distance_metric == "euclidean":
#             return self._euclidean_distance(vec1, vec2)
#         elif self.distance_metric == "dot":
#             return self._dot_product(vec1, vec2)
#         else:
#             raise ValueError(f"Unsupported distance metric: {self.distance_metric}")


# Changed the method call to self.compute_similarity
# # def search(
#         self,
#         query_vector: np.array,
#         k: int
#     ) -> List[Tuple[str, float]]:
#         scores = [
#             (key, self._compute_similarity(query_vector, vector))
#             for key, vector in self.vectors.items()
#         ]
#         return sorted(scores, key=lambda x: x[1], reverse=True)[:k]

# Removed distance_measure parameter
# def search_by_text(
#     self,
#     query_text: str,
#     k: int,
#     return_as_text: bool = False,
# ) -> List[Tuple[str, float]]:
#     query_vector = self.embedding_model.get_embedding(query_text)
#     results = self.search(query_vector, k)
#     return [result[0] for result in results] if return_as_text else results

########  Code changes in Pythonic_RAG_Assignment.ipynb ########
# vector_db = VectorDatabase(distance_metric="euclidean")
# vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

# query = "What is the Michael Eisner Memorial Weak Executive Problem?"
# # query = "What does the text say about management styles?"
# k = 3

# for metric in ["cosine", "euclidean", "dot"]:
#     print(f"\n Metric: {metric.upper()}")
#     print("-" * 40)

#     vector_db = VectorDatabase(distance_metric=metric)
#     vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))  # this was already in your code

#     results = vector_db.search_by_text(query, k=k)
#     for i, (text, score) in enumerate(results, 1):
#         print(f"{i}. Score: {score:.4f}")
#         print(f"   Text: {text[:150].strip()}...\n")



